In [ ]:
# Code from: https://github.com/sintel-dev/Orion/issues/221

In [13]:
def load_anomalies(filename, signal, edges=False):
    anomalies = pd.read_csv(filename)
    anomalies = anomalies.set_index('signal').loc[signal].values[0]
    anomalies = pd.DataFrame(json.loads(anomalies), columns=['start', 'end'])

    if edges:
        data = download(signal)
        start = data.timestamp.min()
        end = data.timestamp.max()

        anomalies['score'] = 1
        parts = np.concatenate([
            [[start, anomalies.start.min(), 0]],
            anomalies.values,
            [[anomalies.end.max(), end, 0]]
        ], axis=0)
        anomalies = pd.DataFrame(parts, columns=['start', 'end', 'score'])

    return anomalies

In [ ]:
from orion import Orion
from orion.primitives.tadgan import TadGAN
from orion.evaluation.contextual import contextual_confusion_matrix
import orion.data as od
import numpy as np
import pandas as pd
import os
import json
from tqdm import tqdm

SIGNALS_SMAP = (
'P-1', 'S-1', 'E-1', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E-7',
	'E-8', 'E-9', 'E-10', 'E-11', 'E-12', 'E-13', 'A-1', 'D-1', 'P-3',
	'D-2', 'D-3', 'D-4', 'A-2', 'A-3', 'A-4', 'G-1', 'G-2', 'D-5',
	'D-6', 'D-7', 'F-1', 'P-4', 'G-3', 'T-1', 'T-2', 'D-8', 'D-9',
	'F-2', 'G-4', 'T-3', 'D-11', 'D-12', 'B-1', 'G-6', 'G-7', 'P-7',
	'R-1', 'A-5', 'A-6', 'A-7', 'D-13', 'A-8', 'A-9', 'F-3'
)

SIGNALS_SML = (
'M-6', 'M-1', 'M-2', 'S-2', 'P-10', 'T-4', 'T-5', 'F-7', 'M-3', 'M-4' 
	'M-5', 'P-15', 'C-1', 'C-2', 'T-12', 'T-13', 'F-4', 'F-5', 'D-14', 
	'T-9', 'P-14', 'T-8', 'P-11', 'D-15', 'D-16', 'M-7', 'F-8'
)

DATAPATH = 'csv/'


with open('tadgan_msl.json') as f:
	hyperparameters = json.load(f)

results = np.zeros([len(SIGNALS_SML), 4]) #[tn, fp, fn, tp]
for i, signal in tqdm(enumerate(SIGNALS_SML)):

	orion = Orion(pipeline = 'tadgan.json', hyperparameters = hyperparameters['init_params'])
	
	train_data = pd.read_csv(DATAPATH + signal +'-train.csv')
	test_data = pd.read_csv(DATAPATH + signal +'-test.csv')
	true_anomalies = load_anomalies('labels.csv', signal)
	orion.fit(train_data)

	anomalies = orion.detect(test_data)
		
	results[i, :] = contextual_confusion_matrix(true_anomalies, anomalies, weighted=False)
	
	np.save('confusion_matrix.npy', results)
			
print("f1 score is" + str(np.sum(results[:, 3])/(np.sum(results[:, 3]) + .5*(np.sum(results[:, 1]) + np.sum(results[:, 2])))))